# COVID-19 Data Exploration (Global and Country-Level)
Exploration of various COVID-19 metrics, including cases, deaths, recoveries, vaccinations, stringency index, etc.

---
## Project Overview
This exploration aims to visually demonstrate the spread of COVID-19 over the first 2-3 years of the pandemic, as well as explore key metrics on both global and country-levels.

### Datasets (GitHub links):
- [Country-Level Dataset](https://github.com/jamesinjune/COVID_19_Data_Exploration/blob/main/visualization_data/covid_daily_country.zip)
- [Global Dataset](https://github.com/jamesinjune/COVID_19_Data_Exploration/blob/main/visualization_data/covid_daily_global.csv)

### Data Sources and Collection:
- Raw data can be found [here]().
- Data cleaning notebooks can be found [here](https://github.com/jamesinjune/COVID_19_Data_Exploration/tree/main/notebooks).
- Original data sources are as follows:
    - https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
    - https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv
    - https://github.com/OxCGRT/covid-policy-tracker/blob/master/data/timeseries/stringency_index_avg.csv
    - https://data.worldbank.org/indicator/SP.POP.TOTL
    - https://hdr.undp.org/data-center/documentation-and-downloads

### Acknowledgements:
- Concept and visualizations inspired by [this notebook](https://www.kaggle.com/code/imdevskp/covid-19-analysis-visualization-comparisons/notebook) on Kaggle. However, this project delves deeper into examining data from individual countries, as well as various factors at play in affecting the spread of COVID across the world.

### Importing Libraries:

In [1]:
import ipywidgets as widgets
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
import seaborn as sns
import wget
import zipfile

from datetime import datetime
from datetime import timedelta
from matplotlib import pyplot as plt
from matplotlib import ticker as ticker
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected=True)

In [2]:
sns.set()
sns.set_palette("colorblind")

# 1. Load Datasets

In [6]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=c773579599c57bc16c626f7f1c7dfd6402d532a0f8bf151a940d40f2ad836262
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [15]:
# Retrieving covid_daily_country.zip from Github
zip_url = "https://github.com/jamesinjune/COVID_19_Data_Exploration/raw/main/visualization_data/covid_daily_country.zip"
wget.download(zip_url, 'covid_daily_country.zip')

with zipfile.ZipFile('covid_daily_country.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [16]:
country_path_import = r"/content/covid_daily_country.csv"
df_country_csv = pd.read_csv(country_path_import, encoding="latin-1")

global_path_import = r"https://github.com/jamesinjune/COVID_19_Data_Exploration/raw/main/visualization_data/covid_daily_global.csv"
df_global_csv = pd.read_csv(global_path_import)

In [17]:
df_country = df_country_csv.copy()
df_global = df_global_csv.copy()

In [18]:
df_country.head()

,country,date,cases,new_cases_smoothed,new_cases_growth_rate,deaths,new_deaths_smoothed,recovered,new_recovered_smoothed,people_vaccinated,...,population,stringency_value,hdi_value,active,infection_rate,people_vaccinated_rate,fully_vaccinated_rate,case_incidence_rate,case_fatality_rate,active_case_rate
0,Afghanistan,2020-01-22,0,0,NaN,0.0,0.0,0.0,NaN,NaN,...,38972232,0.0,0.488,0.0,0.0,NaN,NaN,0.0,NaN,0.0
1,Afghanistan,2020-01-23,0,0,NaN,0.0,0.0,0.0,0.0,NaN,...,38972232,0.0,0.488,0.0,0.0,NaN,NaN,0.0,NaN,0.0
2,Afghanistan,2020-01-24,0,0,NaN,0.0,0.0,0.0,0.0,NaN,...,38972232,0.0,0.488,0.0,0.0,NaN,NaN,0.0,NaN,0.0
3,Afghanistan,2020-01-25,0,0,NaN,0.0,0.0,0.0,0.0,NaN,...,38972232,0.0,0.488,0.0,0.0,NaN,NaN,0.0,NaN,0.0
4,Afghanistan,2020-01-26,0,0,NaN,0.0,0.0,0.0,0.0,NaN,...,38972232,0.0,0.488,0.0,0.0,NaN,NaN,0.0,NaN,0.0


In [19]:
df_global.head()

,date,cases,deaths,recovered,new_cases,new_deaths,new_recovered,active,case_fatality_rate
0,2020-01-22,557,17,30.0,0,0,NaN,510.0,3052.064632
1,2020-01-23,657,18,32.0,50,0,2.0,607.0,2739.726027
2,2020-01-24,944,26,39.0,129,3,4.0,879.0,2754.237288
3,2020-01-25,1437,42,42.0,218,6,4.0,1353.0,2922.755741
4,2020-01-26,2120,56,56.0,313,8,6.0,2008.0,2641.509434


In [ ]:
# Set `date` columns to `datetime` dtype:
df_country["date"] = pd.to_datetime(df_country["date"])
df_global["date"] = pd.to_datetime(df_global["date"])

In [ ]:
# Sort values by `country`, `date`:
df_country = df_country.sort_values(["country", "date"]).reset_index(drop=True)
df_global = df_global.sort_values(["date"]).reset_index(drop=True)

In [ ]:
df_country.head()

In [ ]:
df_global.tail()

# 2. Charts

In [ ]:
country_list = df_country["country"].unique()

## a. Global Charts

In [ ]:
def graph_area_global(measure, color, title):
    df = df_global[["date", measure]].dropna()
    fig = px.area(
        df,
        x="date",
        y=measure,
        color_discrete_sequence=[color],
        hover_data={"date": True, measure: True},
    )
    fig.update_layout(
        title=title,
        xaxis_title="date",
        yaxis_title="count",
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()

In [ ]:
# Global case count
graph_area_global("cases", "#6f6fe7", "Global Cases")

In [ ]:
# Global death count
graph_area_global("deaths", "#ec1342", "Global Deaths")

In [ ]:
# Global recovery count
graph_area_global("recovered", "#12ed5d", "Global Recoveries")

In [ ]:
# Global active count
graph_area_global("active", "#ff9c00", "Global Active")

In [ ]:
# Global new cases
graph_area_global("new_cases", "#6f6fe7", "Global Daily New Cases")

In [ ]:
# Global new deaths
graph_area_global("new_deaths", "#ec1342", "Global Daily New Deaths")

In [ ]:
# Global new recoveries
graph_area_global("new_recovered", "#11de57", "Global Daily New Recoveries")

In [ ]:
# Global cases split by active, deaths, and recovered (stacked)
df_glob_filtered = df_global.dropna(subset=["active"])[
    ["date", "recovered", "deaths", "active"]
]
df_glob_melted = df_glob_filtered.melt(
    id_vars="date", var_name="Measure", value_name="val"
)

fig = px.area(
    df_glob_melted,
    x="date",
    y="val",
    color="Measure",
    color_discrete_sequence=["#10cf51", "#ec1342", "#ff9c00"],
)
fig.update_layout(
    title="Global Total Case Count Split by Active, Deaths, Recoveries",
    xaxis_title="date",
    yaxis_title="count",
    xaxis_rangeslider_visible=True,
    width=800,
    height=600,
)

fig.show()

In [ ]:
# Global case fatality rate
fig = px.line(
    df_global, x="date", y="case_fatality_rate", color_discrete_sequence=["#b50f33"]
)
fig.update_layout(
    title="Global Case Fatality Rate per 100,000 population",
    xaxis_title="date",
    yaxis_title="rate (per 100,000)",
    xaxis_rangeslider_visible=True,
    width=800,
    height=600,
)

fig.show()

## b. Country Charts

In [ ]:
def country_area_graph(country, measure, color, title):
    df = df_country.dropna(subset=[measure])
    df = df[df["country"] == country]
    fig = px.area(df, x="date", y=measure, color_discrete_sequence=[color])
    fig.update_layout(
        title=title,
        xaxis_title="date",
        yaxis_title="count",
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()


country_dropdown = widgets.Dropdown(
    options=country_list,
    description="Country:",
)

In [ ]:
# Country total case count
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("cases"),
    color=widgets.fixed("#6f6fe7"),
    title=widgets.fixed("Total Case Count by Country"),
)

In [ ]:
# Country total death count
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("deaths"),
    color=widgets.fixed("#ec1342"),
    title=widgets.fixed("Total Death Count by Country"),
)

In [ ]:
# Country total recovered count
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("recovered"),
    color=widgets.fixed("#11de57"),
    title=widgets.fixed("Total Recovery Count by Country"),
)

In [ ]:
# Country total active count
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("active"),
    color=widgets.fixed("#ff9c00"),
    title=widgets.fixed("Total Active Case Count by Country"),
)

In [ ]:
# Country daily new cases
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("new_cases_smoothed"),
    color=widgets.fixed("#6f6fe7"),
    title=widgets.fixed("Daily New Cases by Country"),
)

In [ ]:
# Country daily new cases
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("new_deaths_smoothed"),
    color=widgets.fixed("#ec1342"),
    title=widgets.fixed("Daily New Deaths by Country"),
)

In [ ]:
# Country daily new recovered
widgets.interactive(
    country_area_graph,
    country=country_dropdown,
    measure=widgets.fixed("new_recovered_smoothed"),
    color=widgets.fixed("#11de57"),
    title=widgets.fixed("Daily New Recoveries by Country"),
)

In [ ]:
# Country total case count split by active, deaths, recoveries (stacked)
def country_case_stacked_graph(country):
    df = df_country[df_country["country"] == country].set_index("country")
    df_filtered = df.dropna(subset=["active"])[
        ["date", "recovered", "deaths", "active"]
    ]
    df_melted = df_filtered.melt(id_vars="date", var_name="Measure", value_name="val")

    fig = px.area(
        df_melted,
        x="date",
        y="val",
        color="Measure",
        color_discrete_sequence=["#10cf51", "#ec1342", "#ff9c00"],
    )
    fig.update_layout(
        title=f"Total Case Count Split by Active, Deaths, Recoveries in {country}",
        xaxis_title="date",
        yaxis_title="count",
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()


widgets.interact(country_case_stacked_graph, country=country_list)

In [ ]:
# Country vaccination counts (filterable)
vaccination_measure_list = [
    "people_vaccinated",
    "people_fully_vaccinated",
    "total_vaccinations",
    "total_boosters",
    "daily_people_vaccinated",
    "daily_people_fully_vaccinated",
    "daily_vaccinations",
    "daily_boosters",
]


def graph_country_vaccination(country, measure):
    df = df_country[df_country["country"] == country]
    df = df[
        [
            "date",
            "people_vaccinated",
            "people_fully_vaccinated",
            "total_vaccinations",
            "total_boosters",
            "daily_people_vaccinated",
            "daily_people_fully_vaccinated",
            "daily_vaccinations",
            "daily_boosters",
        ]
    ]
    fig = px.area(df, x="date", y=measure, color_discrete_sequence=["#6ad2e5"])
    fig.update_layout(
        title=f"{measure} in {country}",
        xaxis_title="date",
        yaxis_title=f"{measure}",
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()


widgets.interact(
    graph_country_vaccination, country=country_list, measure=vaccination_measure_list
)

In [ ]:
# Country stringency index over time
def graph_country_stringency(country):
    df = df_country[df_country["country"] == country]
    df = df[["date", "stringency_value"]].dropna()
    fig = px.line(
        df, x="date", y="stringency_value", color_discrete_sequence=["#d97670"]
    )
    fig.update_layout(
        title=f"Stringency Index in {country}",
        xaxis_title="date",
        yaxis_title="stringency index",
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()


widgets.interact(graph_country_stringency, country=country_list)

In [ ]:
def graph_country_rates(country, measure, color):
    df = df_country[df_country["country"] == country]
    df = df[["date", measure]].dropna()
    fig = px.line(df, x="date", y=measure, color_discrete_sequence=[color])
    fig.update_layout(
        title=f"{measure} in {country}",
        xaxis_title="date",
        yaxis_title=measure,
        xaxis_rangeslider_visible=True,
        width=800,
        height=600,
    )
    fig.show()

In [ ]:
# Country infection rate over time
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("infection_rate"),
    color=widgets.fixed("#d97670"),
)

In [ ]:
# Country active case ratio over time
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("active_case_rate"),
    color=widgets.fixed("#ff9c00"),
)

In [ ]:
# Country people vaccination ratio over time
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("people_vaccinated_rate"),
    color=widgets.fixed("#6ad2e5"),
)

In [ ]:
# Country fully vaccinated ratio over time
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("fully_vaccinated_rate"),
    color=widgets.fixed("#6ad2e5"),
)

In [ ]:
# Country case fatality rates
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("case_fatality_rate"),
    color=widgets.fixed("#d97670"),
)

In [ ]:
# Country case incidence rates
widgets.interactive(
    graph_country_rates,
    country=country_list,
    measure=widgets.fixed("case_incidence_rate"),
    color=widgets.fixed("#d97670"),
)

In [ ]:
# Create date slider widget for graphs below
def create_date_slider(measure):
    start_date = df_country.dropna(subset=measure)["date"].iloc[0]
    end_date = df_country.dropna(subset=measure)["date"].iloc[-1]
    interval = timedelta(days=1)

    # Create date range
    date_range = []
    current_date = start_date
    while current_date <= end_date:
        date_range.append(current_date)
        current_date += interval

    # Create a slider widget
    date_slider = widgets.SelectionSlider(
        options=[(d.strftime("%Y-%m-%d"), d) for d in date_range],
        value=start_date,
        description="Select Date:",
        continuous_update=False,
    )
    return date_slider


# Creates bar graphs of top/bottom 15 countries in given measure
def graph_country_bar(measure, date, is_top_n=True, hdi_colored=False):
    df = df_country[df_country["population"] > 1000000]
    df = df[df["date"] == date]
    df = df[["country", measure, "hdi_value"]]
    if is_top_n == True:
        df = (
            df.sort_values(by=measure, ascending=False)
            .head(15)
            .sort_values(by=measure, ascending=True)
        )
    else:
        df = (
            df.sort_values(by=measure, ascending=True)
            .head(15)
            .sort_values(by=measure, ascending=False)
        )
    df["hdi_value"] = df["hdi_value"].round(2)

    if hdi_colored == True:
        fig = px.bar(
            df,
            x=measure,
            y="country",
            orientation="h",
            color="hdi_value",
            range_color=[0, 1],
            hover_data={"hdi_value": True},
        )
    else:
        fig = px.bar(
            df,
            x=measure,
            y="country",
            orientation="h",
            range_color=[0, 1],
            hover_data={"hdi_value": True},
        )
    if is_top_n == True:
        fig.update_layout(
            title=f"Top 15 Countries by {measure}",
            xaxis_title="country",
            yaxis_title=measure,
            width=800,
            height=600,
        )
    else:
        fig.update_layout(
            title=f"Bottom 15 Countries by {measure}",
            xaxis_title="country",
            yaxis_title=measure,
            width=800,
            height=600,
        )
    fig.show()

In [ ]:
# Bar charts of top 15 countries by infection rate, colored in hdi
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("infection_rate"),
    date=create_date_slider("infection_rate"),
    is_top_n=True,
    hdi_colored=widgets.fixed(True),
)

In [ ]:
# Bar charts of top 15 countries by cases
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("cases"),
    date=create_date_slider("cases"),
    is_top_n=True,
    hdi_colored=widgets.fixed(False),
)

In [ ]:
# Bar charts of top 15 countries by deaths
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("deaths"),
    date=create_date_slider("deaths"),
    is_top_n=True,
    hdi_colored=widgets.fixed(False),
)

In [ ]:
# Bar charts of top 15 countries by active
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("active"),
    date=create_date_slider("active"),
    is_top_n=True,
    hdi_colored=widgets.fixed(False),
)

In [ ]:
# Bar chart of top 15 countries by vaccinations
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("people_vaccinated"),
    date=create_date_slider("people_vaccinated"),
    is_top_n=True,
    hdi_colored=widgets.fixed(False),
)

In [ ]:
# Bar charts of top 15 countries by fully vaccinated people
widgets.interactive(
    graph_country_bar,
    measure=widgets.fixed("people_fully_vaccinated"),
    date=create_date_slider("people_fully_vaccinated"),
    is_top_n=True,
    hdi_colored=widgets.fixed(False),
)

In [ ]:
def graph_country_dual(country, measure_y1, measure_y2):
    df = df_country[df_country["country"] == country]
    df = df[["date", measure_y1, measure_y2]].dropna()

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["date"],
            y=df[measure_y1],
            fill="tozeroy",
            mode="lines",
            name=f"{measure_y1}",
            line=dict(color="#6f6fe7"),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["date"],
            y=df[measure_y2],
            mode="lines",
            name=f"{measure_y2}",
            yaxis="y2",
            line=dict(color="#d97670"),
        )
    )
    fig.update_layout(
        title="Dual-Axis Line Chart",
        xaxis_title="date",
        yaxis_title=f"{measure_y1}",
        yaxis2=dict(title=f"{measure_y2}", overlaying="y", side="right"),
        xaxis_rangeslider_visible=True,
        width=1000,
        height=600,
    )
    fig.show()

In [ ]:
# New cases and stringency index by country
widgets.interactive(
    graph_country_dual,
    country=country_list,
    measure_y1=widgets.fixed("new_cases_smoothed"),
    measure_y2=widgets.fixed("stringency_value"),
)

### c. Scatterplots:

In [ ]:
# Boxplot of hdi distribution
def hdi_dist(date):
    df = df_country[df_country["date"] == date]
    fig = px.box(df, x="hdi_value")
    fig.update_layout(width=1000, height=400)
    fig.show()


widgets.interact(hdi_dist, date=create_date_slider("hdi_value"))

In [ ]:
def graph_scatter(measure_x, measure_y, date, log_x=False, log_y=False):
    df = df_country[df_country["date"] == date]
    df = df[df["population"] > 1000000]
    df["date"] = df["date"].astype(str)
    df[measure_x] = df[measure_x] + 1
    df[measure_y] = df[measure_y] + 1

    fig = px.scatter(
        df,
        x=measure_x,
        y=measure_y,
        trendline="ols",
        trendline_options=dict(log_x=log_x, log_y=log_y),
        hover_data={"country": True, "date": True},
        log_x=log_x,
        log_y=log_y,
    )
    fig.update_layout(width=1000, height=400)
    fig.show()

In [ ]:
# Scatterplot of hdi vs case fatality rate
widgets.interactive(
    graph_scatter,
    measure_x=widgets.fixed("hdi_value"),
    measure_y=widgets.fixed("case_fatality_rate"),
    date=create_date_slider("hdi_value"),
    log_x=widgets.fixed(False),
    log_y=widgets.fixed(True),
)

In [ ]:
# Scatterplot of hdi vs infection rate
widgets.interactive(
    graph_scatter,
    measure_x=widgets.fixed("hdi_value"),
    measure_y=widgets.fixed("infection_rate"),
    date=create_date_slider("hdi_value"),
    log_x=widgets.fixed(False),
    log_y=widgets.fixed(True),
)

In [ ]:
# Scatterplot of population vs infection rate
widgets.interactive(
    graph_scatter,
    measure_x=widgets.fixed("population"),
    measure_y=widgets.fixed("infection_rate"),
    date=create_date_slider("population"),
    log_x=widgets.fixed(True),
    log_y=widgets.fixed(True),
)

In [ ]:
# Scatterplot of infection rate vs vaccinations
widgets.interactive(
    graph_scatter,
    measure_x=widgets.fixed("people_vaccinated_rate"),
    measure_y=widgets.fixed("infection_rate"),
    date=create_date_slider("people_vaccinated_rate"),
    log_x=widgets.fixed(False),
    log_y=widgets.fixed(True),
)

In [ ]:
# Scatterplot of infection rate vs vaccinations
widgets.interactive(
    graph_scatter,
    measure_x=widgets.fixed("fully_vaccinated_rate"),
    measure_y=widgets.fixed("infection_rate"),
    date=create_date_slider("fully_vaccinated_rate"),
    log_x=widgets.fixed(False),
    log_y=widgets.fixed(True),
)